# Import Libraries

In [ ]:
import numpy as np
import pandas as pd 
import os
import requests
import random
random.seed(6)
#import catboost 
#from catboost import CatBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, RobustScaler
#from sklearn.model_selection import KFold,GroupKFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split, cross_val_score
#from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score
from numpy import mean, std
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load Data

In [ ]:
df = pd.read_csv('../input/phase-ii-widsdatathon2022/ccai/ccai/data.csv')
df.head(1)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(col, df[col].nunique())

# Data Pre-processing

In [ ]:
# Preprocess City Names. Necessary or not?

df.city_name[df.city_name.str.contains('MARSEILLE')]='MARSEILLE'
df.city_name[df.city_name.str.contains('PARIS')]='PARIS'

In [ ]:
df.city_name.nunique()

# Numerical Features

In [ ]:
numeric_cols = df.columns[(df.dtypes=='int')|(df.dtypes=='float')]
numeric_cols

In [ ]:
df['city_counts'] = df.groupby('city_name')['city_name'].transform('count')

In [ ]:
df.describe()

In [ ]:
corr = df[numeric_cols].corr()
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, ax=ax)

In [ ]:
corr['consumption'].sort_values()

# City-Level Features

In [ ]:
big_city = df[df.city_counts>999]
for city in big_city.city_name.unique():
    city_df = df[df.city_name==city]
    city_df = city_df.add_suffix(city)
    print(city_df.corr().abs()['consumption'+city].sort_values(ascending=False))
    print()

# Feature Plots

In [ ]:
df.plot.scatter('delivery_points','consumption')

In [ ]:
df.plot.scatter('height','consumption')

In [ ]:
df.plot.scatter('floors','consumption')

In [ ]:
df.plot.scatter('alt_prec','consumption')

In [ ]:
df.plot.scatter('wall_mat','consumption')

In [ ]:
df.plot.scatter('roof_mat','consumption')

In [ ]:
df.plot.scatter('wall_mat','roof_mat')

In [ ]:
df.plot.scatter('floors','height')

# Categorical Features

In [ ]:
# Drop all rows with any NaN values. Necessary or Not?
#df = df.dropna(axis=0)

In [ ]:
cat_cols = list(df.columns[df.dtypes=='object'])
for col in cat_cols:
    cat_df = (df.groupby(col).mean()) # .reset_index()
    corr = cat_df.corr().abs()
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_title('Data grouped by '+ col)
    sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, ax=ax)
    print(corr['consumption'].sort_values(ascending=False), plt.show())
    print('Data grouped by '+ col)

# Data grouped by type

In [ ]:
df.type.nunique()

In [ ]:
df.type.value_counts()

In [ ]:
df_grouped_by_type = (df.groupby('type').mean()) # .reset_index()
corr = df_grouped_by_type.corr().abs()
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('Data grouped by type')
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, ax=ax)
print(corr['consumption'].sort_values(ascending=False), plt.show())
print( 'Data grouped by type')

In [ ]:
df_grouped_by_type